In [215]:
import obspy
import struct
import numpy as np
import math

def _bcd(byte):
    """Decode 1-byte binary code decimals."""

    if isinstance(byte, str):
        try:
            byte = ord(byte)
        except TypeError:
            raise ValueError('not a byte')
    elif isinstance(byte, int):
        if byte > 255:
            raise ValueError('not a byte')

    else:
        raise ValueError('not a byte')
    v1 = byte >> 4
    v2 = byte & 0xF
    return v1, v2

def _decode_bcd(bytes_in):
    """Decode arbitrary length binary code decimals."""
    v = 0
    if isinstance(bytes_in, int):
        bytes_in = bytes([bytes_in])
        disp('bytes in')
    n = len(bytes_in)
    n = n*2 - 1  # 2 values per byte
    for byte in bytes_in:
        v1, v2 = _bcd(byte)
        v += v1*10**n + v2*10**(n-1)
        n -= 2
    return v

def _decode_bin(bytes_in):
    """Decode unsigned ints."""
    if isinstance(bytes_in, int):
        bytes_in = bytes([bytes_in])
    ll = len(bytes_in)
    # zero-pad to 4 bytes
    b = (chr(0)*(4-ll)).encode()
    b += bytes_in
    return struct.unpack('>I', b)[0]

def _decode_flt(bytes_in):
    """Decode single-precision floats."""
    if isinstance(bytes_in, int):
        bytes_in = bytes([bytes_in])
    ll = len(bytes_in)
    # zero-pad to 4 bytes
    b = (chr(0)*(4-ll)).encode()
    b += bytes_in
    f = struct.unpack('>f', b)[0]
    if math.isnan(f):
        f = None
    return f

In [3]:
smpl_file = '.\data\STRYDE_SEGDv3_Read_Pack_v1.4continuous_data_example_1trace.rsamp'
# with open(smpl_file, mode='rb') as file: # b is important -> binary
#     fileContent = file.read()


In [4]:
fp = open(smpl_file, 'rb')
buf_header1 = fp.read(32)
buf_header2 = fp.read(32)
buf_header3 = fp.read(32)
fp.close()

In [132]:
general_header1 = dict()
if _decode_bcd(buf_header1[0:2]) > 9999:
    general_header1['file number'] = '{0:0{1}x}'.format(buf_header1[0], 2) + '{0:0{1}x}'.format(buf_header1[1], 2)
else:
    general_header1['file number'] = _decode_bcd(buf_header1[0:2])

general_header1['format code'] = _decode_bcd(buf_header1[2:4])

_year = _decode_bcd(buf_header1[10:11]) + 2000
_nblocks, _jday = _bcd(buf_header1[11])
general_header1['n_additional_blocks'] = _nblocks   # important
_jday *= 100
_jday += _decode_bcd(buf_header1[12:13])
_hour = _decode_bcd(buf_header1[13:14])
_min = _decode_bcd(buf_header1[14:15])
_sec = _decode_bcd(buf_header1[15:16])
general_header1['time'] = obspy.UTCDateTime(year=_year, julday=_jday,
                               hour=_hour, minute=_min, second=_sec)

# 여기 사이에 추가로 더 들어가야 함.
general_header1['scan_type_per_record'] = _decode_bcd(buf_header1[27:28])
general_header1['n_channel_sets_per_scan_type'] = _decode_bcd(buf_header1[28:29])
general_header1['n_sample_skew_32bit_extensions'] = _decode_bcd(buf_header1[29:30])
if _decode_bcd(buf_header1[30:31]) > 99:
    general_header1['extended_header_length'] = 'ff'
else:
    general_header1['extended_header_length'] = _decode_bcd(buf_header1[30:31])
general_header1['external_header_length'] = _decode_bcd(buf_header1[31:32])



In [133]:
general_header1

{'file number': 'ffff',
 'format code': 8036,
 'n_additional_blocks': 3,
 'time': 2020-04-21T15:00:00.000000Z,
 'scan_type_per_record': 1,
 'n_channel_sets_per_scan_type': 5,
 'n_sample_skew_32bit_extensions': 0,
 'extended_header_length': 'ff',
 'external_header_length': 0}

In [232]:
# 이거는 거의 완성
general_header2 = dict()

general_header2['extended_file_number'] = _decode_bin(buf_header2[0:3])
general_header2['extended_channel_sets_per_scan_types'] = _decode_bin(buf_header2[3:5])
general_header2['extended_header_blocks'] = _decode_bin(buf_header2[5:8])
general_header2['external_header_blocks'] = _decode_bin(buf_header2[8:10]) # or extended skew blocks

_rev = ord(buf_header2[10:11])
_rev += ord(buf_header2[11:12])/10.
general_header2['segd_revision_number'] = _rev
general_header2['no_blocks_of_general_trailer'] = _decode_bin(buf_header2[12:16])
general_header2['extended_record_length_in_ms'] = _decode_bin(buf_header2[16:20])
general_header2['record_set_number'] = _decode_bin(buf_header2[20:22])
general_header2['n_extended_additional_blocks'] = _decode_bin(buf_header2[22:24])
general_header2['dominant_sampling_interval_in_us'] = _decode_bin(buf_header2[24:27])
general_header2['external_header_blocks'] = _decode_bin(buf_header2[27:30])
general_header2['header_block_type'] = _decode_bin(buf_header2[31:32])

In [233]:
general_header2

{'extended_file_number': 10002,
 'extended_channel_sets_per_scan_types': 5,
 'extended_header_blocks': 374,
 'external_header_blocks': 0,
 'segd_revision_number': 3.0,
 'no_blocks_of_general_trailer': 0,
 'extended_record_length_in_ms': 3659000000,
 'record_set_number': 0,
 'n_extended_additional_blocks': 3,
 'dominant_sampling_interval_in_us': 2000,
 'header_block_type': 2}

In [ ]:
fp = open(smpl_file, 'rb')
buf_header1 = fp.read(32)
buf_header2 = fp.read(32)
buf_header3 = fp.read(32)
fp.close()

In [95]:
print(buf_header2[5:8])
print(buf_header2[8:9])

b'\x00\x01v'
b'\x00'


In [158]:
# channel import
buf_channel = dict()
buf_channel = []
fp = open(smpl_file, 'rb')
fp.seek((general_header1['n_additional_blocks'] + 1) * 32)
for i in range(general_header1['n_channel_sets_per_scan_type']):
    # buf_channel[str(i+1)] = fp.read(96)
    buf_channel.append(fp.read(96))

fp.close()

In [225]:
i = 0
channel = dict()
channel['scan_type'] = _decode_bcd(buf_channel[i][0:1])
channel['channel_set_number'] = _decode_bin(buf_channel[i][1:3])
channel['channel_type_identification'] = '{0:0{1}x}'.format(buf_channel[i][3], 2)   # hex
channel['channel_set_start_time'] = _decode_bcd(buf_channel[i][4:8])
channel['channel_set_end_time'] = _decode_bcd(buf_channel[i][8:12])
channel['n_samples_in_each_trac_of_this_channel_set'] = _decode_bin(buf_channel[i][12:16])
channel['sample_descale_multiplication_factor'] = round(_decode_flt(buf_channel[i][16:20]), 5) # Careful this
channel['n_channels_in_this_channel_set'] = _decode_bin(buf_channel[i][20:23])
channel['sampling_interval_in_us'] = _decode_bin(buf_channel[i][23:26])



# Extended header

In [238]:

fp = open(smpl_file, 'rb')
fp.seek((general_header1['n_additional_blocks'] + 1) * 32 +
        general_header1['n_channel_sets_per_scan_type'] * 96)

test = fp.read(general_header2['extended_header_blocks'] * 32)

fp.close()
test

b'{\n "sync": {\n  "syncEventInfo": [\n   {\n    "syncEvent": "eSYNC_EVENT_OPTICAL_LINK",\n    "localTimeInTicks": "5119000000000",\n    "globalTime": {\n     "tvSec": "1271514446",\n     "tvNsec": 875002000,\n     "timeType": "eTIME_TYPE_GPS"\n    },\n    "opticalSyncSource": "eNODE_TYPE_NEST_NODE"\n   },\n   {\n    "syncEvent": "eSYNC_EVENT_OPTICAL_LINK",\n    "localTimeInTicks": "5119999000000",\n    "globalTime": {\n     "tvSec": "1271516398",\n     "tvNsec": 46873000,\n     "timeType": "eTIME_TYPE_GPS"\n    },\n    "opticalSyncSource": "eNODE_TYPE_MOC"\n   },\n   {\n    "syncEvent": "eSYNC_EVENT_INTERNAL_GNSS",\n    "localTimeInTicks": "5120003584000",\n    "globalTime": {\n     "timeType": "eTIME_TYPE_GPS"\n    },\n    "NmeaString": "@UBX BIN::hdr: 0xb5 62 class: 0x1 id: 0x20 length: 16\\n\\t ::   GPS:: iTOW: 226807000 fTOW: 0 week: 2102 leapS: 18 valid: 7 tACC: 0",\n    "ubxMsgBytes": "tWIBIBAA2MyEDQAAAAA2CBIHAAAAAL1g"\n   },\n   {\n    "syncEvent": "eSYNC_EVENT_INTERNAL_GNSS",\

# Data

In [239]:
fp = open(smpl_file, 'rb')
fp.seek((general_header1['n_additional_blocks'] + 1) * 32 +
        general_header1['n_channel_sets_per_scan_type'] * 96 +
        general_header2['extended_header_blocks'] * 32)

test = fp.read(20)

fp.close()
test

b"\xff\xff\x01\x01\x00\x01\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00'\x12"

In [ ]:
trace_header = dict()
if _decode_bcd(test[0:2]) > 9999:
    trace_header['file_number'] = '{0:0{1}x}'.format(buf_header1[0], 2) + '{0:0{1}x}'.format(buf_header1[1], 2)
else:
    trace_header['file_number'] = _decode_bcd(test[0:2])

In [243]:
_decode_bcd(test[2:3])
_decode_bcd(test[0:2]) > 9999

True

In [245]:
test[1]

255

In [129]:
(general_header1['n_additional_blocks'] + 1) * 32

128

In [102]:
_decode_bin(buf_header2[12:16])

0

In [60]:
_ehl = _decode_bcd(buf_header1[31:32])
_ehl == 0xFF

False

In [62]:
buf_header1[1] == 0xFF

True

In [48]:
_bsi /= 10.

In [52]:
_bsi
_rec_type, _rec_len = _bcd(buf_header1[25])

In [53]:
_rec_type

1

In [54]:
_rec_len

15

In [18]:
# buf_header1[0:2]
buf_header1[0]

255

In [26]:
a = '{0:0{1}x}'.format(buf_header1[0], 2) + '{0:0{1}x}'.format(buf_header1[1], 2)
# '{0:0{1}x}'.format(buf_header1[1], 2)
a

'ffff'